In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 130.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 22.5 MB/s eta 0:00:00


In [ ]:
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes --quiet

In [ ]:
import torch

In [ ]:
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

In [ ]:
from unsloth import FastLanguageModel

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit)

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
text = "I love using Unsloth in Colab!"

In [ ]:
tokens = tokenizer.tokenize(text)
print(f"Tokens: {tokens}")

In [ ]:
input_ids = tokenizer.encode(text)
print(f"IDs:    {input_ids}")

In [ ]:
decoded = tokenizer.decode(input_ids)
print(f"Back:   '{decoded}'")

In [ ]:
print(tokenizer.special_tokens_map)

In [ ]:
print(f"{'ID':<10} {'Token':<20}")
print("-" * 30)

# Loop through all special IDs and decode them
for id_num in tokenizer.all_special_ids:
    # We decode the ID to see the text representation
    token_text = tokenizer.decode([id_num])
    print(f"{id_num:<10} {token_text:<20}")

In [ ]:
# Check specific IDs that we know belong to Llama-3 structure tags
interesting_ids = [128001, 128002, 128003, 128004, 128005, 128006, 128007, 128008, 128009, 128010]

print(f"{'ID':<10} {'Token String':<30}")
print("-" * 40)

for id_num in interesting_ids:
    # Decode strictly to see the text
    token_str = tokenizer.decode([id_num])
    print(f"{id_num:<10} {repr(token_str):<30}")

In [ ]:
print(tokenizer.chat_template)

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "List 3 famous distinct colors of apples."}
]

In [ ]:
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt").to("cuda")

In [ ]:
inputs

In [ ]:
output = model.generate(
    input_ids = inputs,
    max_new_tokens = 128,
    use_cache = True,
    temperature = 1.5,
    min_p = 0.1
)

In [ ]:
output

In [ ]:
decoded_output = tokenizer.batch_decode(output)

In [ ]:
decoded_output[0]

In [ ]:
print(decoded_output[0])

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
    )